Design a data structure that follows the constraints of a Least Recently Used (LRU) cache.

Implement the LRUCache class:

LRUCache(int capacity) Initialize the LRU cache with positive size capacity.
int get(int key) Return the value of the key if the key exists, otherwise return -1.
void put(int key, int value) Update the value of the key if the key exists. Otherwise, add the key-value pair to the cache. If the number of keys exceeds the capacity from this operation, evict the least recently used key.
The functions get and put must each run in O(1) average time complexity.

 

Example 1:

Input
["LRUCache", "put", "put", "get", "put", "get", "put", "get", "get", "get"]
[[2], [1, 1], [2, 2], [1], [3, 3], [2], [4, 4], [1], [3], [4]]
Output
[null, null, null, 1, null, -1, null, -1, 3, 4]

Explanation
LRUCache lRUCache = new LRUCache(2);
lRUCache.put(1, 1); // cache is {1=1}
lRUCache.put(2, 2); // cache is {1=1, 2=2}
lRUCache.get(1);    // return 1
lRUCache.put(3, 3); // LRU key was 2, evicts key 2, cache is {1=1, 3=3}
lRUCache.get(2);    // returns -1 (not found)
lRUCache.put(4, 4); // LRU key was 1, evicts key 1, cache is {4=4, 3=3}
lRUCache.get(1);    // return -1 (not found)
lRUCache.get(3);    // return 3
lRUCache.get(4);    // return 4
 

Constraints:

1 <= capacity <= 3000
0 <= key <= 104
0 <= value <= 105
At most 2 * 105 calls will be made to get and put.

# approach 1:
- to acheive the O(1) tc, you should use insert and delete without bothering of moving the other elements, 
- which can be done using linked list.

In [ ]:
class Node:
    def __init__(self, key: int, val: int):
        self.key = key
        self.val = val
        self.prev = None
        self.next = None

class LRUCache:
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.cache = {}  # key -> node

        # Left = LRU, Right = MRU.
        self.start = Node(0, 0)  # dummy LRU
        self.end = Node(0, 0) # dummy MRU
        self.start.next = self.end
        self.end.prev = self.start

    # Helper to remove a node from the DLL
    def remove(self, node: Node):
        prev_node = node.prev
        next_node = node.next

        prev_node.next = next_node
        next_node.prev = prev_node

    # Helper to insert at right (MRU)
    def insert(self, node: Node):
        prev_node = self.end.prev

        prev_node.next = node
        node.prev = prev_node

        node.next = self.end
        self.end.prev = node

    def get(self, key: int) -> int:
        if key in self.cache:
            node = self.cache[key]
            # remove the node.
            self.remove(node)

            # insert in the MRU, which is the end side.
            self.insert(node)
            return node.val
        return -1

    def put(self, key: int, value: int) -> None:
        # if its present in the hash, we remove it and put it in the back.
        if key in self.cache:
            self.remove(self.cache[key])

        # create the new node with the give value.
        new_node = Node(key, value)
        self.cache[key] = new_node

        # insert at the MRU side. which is the end side,
        self.insert(new_node)

        if len(self.cache) > self.capacity:
            # Remove from the LRU side. which is the start side.
            lru = self.start.next
            self.remove(lru)
            del self.cache[lru.key]
